<H1>Scraping the Wikipedia page to create the Dataframe<h1>

<h3>using BeautifulSoup librarie<h3>

*Import Libraries*

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


*Scrap List of postal codes of Canada*

In [2]:
# download data 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

*Converting the content of Postal Codes table to a dataframe*

In [11]:
# creating a new Dataframe
column_names = ['Postcode','Borough','Neighborhood']
canada = pd.DataFrame(columns = column_names)

# loop to find postcode, borough and neighborhood 
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    canada = canada.append({'Postcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
canada = canada[canada.Borough!='Not assigned']
canada = canada[canada.Borough!= 0]
canada.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,canada.shape[0]):
    if canada.iloc[i][2] == 'Not assigned':
        canada.iloc[i][2] = canada.iloc[i][1]
        i = i+1
                                 
can_df = canada.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
can_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> Cleaning Data<h/>

Droping "None" rows and rows which contains 'Not assigned' value

In [13]:
can_df = can_df.dropna()
empty = 'Not assigned'
can_df = can_df[(can_df.Postcode != empty ) & (can_df.Borough != empty) & (can_df.Neighborhood != empty)]
can_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
gp = can_df.groupby(['Postcode', 'Borough'])
df = gp.apply(neighborhood_list).reset_index(name='Neighborhood')
print(df.shape)
df.head()

(103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df.to_csv('toronto_part1.csv')
len(df['Postcode'])

103